# Building deep learning models with keras
In this notebook, you'll use the Keras library to build deep learning models for both regression and classification. You'll learn about the Specify-Compile-Fit workflow that you can use to make predictions, and by the end of this, you'll have all the tools necessary to build deep neural networks.

#### Model building steps
The Keras workflow has 4 steps. 
1. you specify the architecture, which is things like: how many layers do you want? how many nodes in each layer? What activation function do you want to use in each layer? 

2. you compile the model. This specifies the loss function, and some details about how optimization works. 

3. you fit the model. Which is that cycle of back-propagation and optimization of model weights with your data. 

4. finally you will want to use your model to make predictions. We'll go through these steps sequentially. The first step is creating or specifying your model.

####  Model specification ( Regression model)

In [ ]:
# Import necessary modules
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Read the file
df = pd.read_csv('hourly_wages.csv')

# Separate predictors and target
predictors = df.drop('wage_per_hour', axis=1).values
target = df['wage_per_hour'].values

# Save the number of columns in predictors: n_cols
n_cols = predictors.shape[1]

# Set up the model: model
model = Sequential()

# Add the first layer
model.add(Dense(50, activation="relu", input_shape=(n_cols, )))

# Add the second layer
model.add(Dense(32, activation="relu"))

# Add the output layer
model.add(Dense(1))

First we import what we will need. pandas is here only for reading some data. The other two imports are used for building our model. Then we seperate predictors and target from the dataset.
We always need to specify how many columns (n_cols) are in the input when building a Keras model, because that is the number of nodes in the input layer. 

We then start building the model. The first line of model specification is model equals Sequential. There are two ways to build up a model, and we will focus on sequential, which is the easier way to build a model. Sequential models require that each layer has weights or connections only to the one layer coming directly after it in the network diagram. There are more exotic models out there with complex patterns of connections, but Sequential will do the trick for everything we need here. We start adding layers using the add method of the model. 

The type of layer you have seen, that standard layer type, is called a Dense layer. It is called Dense because all of the nodes in the previous layer connect to all of the nodes in the current layer. As you advance in deep learning, you may start using layers that aren't Dense. In each layer, we specify the number of nodes as the first positional argument, and the activation function we want to use in that layer using the keyword argument activation. Keras supports every activation function you will want in practice. In the first layer, we need to specify input shapes as shown here. That says the input will have n_cols columns, and there is nothing after the comma, meaning it can have any number of rows, that is, any number of data points. 

You'll notice the last layer has 1 node. That is the output layer. This model has 2 hidden layers, and an output layer. You may be struck that hidden layers has 50 and 32 nodes. Keras and TensorFlow do the math for us, so don't feel afraid to use much bigger networks than we've seen before. It's quite common to use 100 or 1000s nodes in a layer. You'll learn more about choosing an appropriate number of nodes later.

#### Compiling a model

After you've specified a model, the next task is to compile it, which sets up the network for optimization, for instance creating an internal function to do back-propagation efficiently.The compile methods has two important arguments for you to choose. 

The first is what optimizer to use, which controls the learning rate. In practice, the right choice of learning rate can make a big difference for how quickly our model finds good weights, and even how good a set of weights it can find. There are a few algorithms that automatically tune the learning rate. Even many experts in the field don't know all the details of all the optimization algorithms. So the pragmatic approach is to choose a versatile algorithm and use that for most problems. Adam is an excellent choice as your go-to optimizer. Adam adjusts the learning rate as it does gradient descent, to ensure reasonable values throughout the weight optimization process. 

The second thing you specify is the loss function. Mean squared error is the most common choice for regression problems. When we use Keras for classification, you will learn a new default metric.


In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

# Verify that model contains information from compiling
print("Loss function: " + model.loss)

#### fit a model
After compiling the model, you can fit it. That is applying back-propagation and gradient descent with your data to update the weights. The fit step looks similar to what you've seen in scikit-learn, though it has more options which we will explore soon. Even with the Adam optimizer, which is pretty smart, it can improve your optimization process if you scale all the data so each feature is, on average, about similar sized values. One common approach is to subtract each feature by that features mean, and divide it by it's standard deviation.

After the compile step, we run fit, with the predictors as the first argument. When you run this, you will see some output showing the optimizations progress as it fits the data.

In [ ]:
# Fit the model
model.fit(predictors, target)

#### Classification Models
So far we have focused on regression models. But deep learning works similarly for classification, that is for predicting outcomes from a set of discrete options.

For classification, you do a couple of things differently. The biggest changes are: first, you set the loss function as 'categorical_crossentropy' instead of 'mean_squared_error'. This isn't the only possible loss function for classification problems, but it is by far the most common. You may have heard of this before under the name LogLoss. We won't go into the mathematical details of categorical crossentropy here. For categorical crossentropy loss function, a lower score is better. But it's still hard to interpret. So I've added this argument "metrics equals accuracy". This means I want to print out the accuracy score at the end of each epoch, which makes it easier to see and understand the models progress.

The second thing we do is you need to modify the last layer, so it has a separate node for each potential outcome. You will also change the activation function to softmax. The softmax activation function ensures the predictions sum to 1, so they can be interpreted as probabilities.

In [ ]:
# Import necessary modules
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


# Read the data
df = pd.read_csv('titanic_all_numeric.csv')

# Separate predictors and target
predictors = df.drop('survived', axis=1).values.astype('float32')

# Convert the target to categorical: target
targets = to_categorical(df.survived)


# Save the number of columns in predictors: n_cols
n_clos = predictors.shape[1]

# set up the model
model = Sequential()

# Add the first layers
model.add(Dense(32, activation='relu', input_shape=(n_clos, )))

# Add the output layers
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=2)

# Fit the model
model.fit(predictors, targets, validation_split=0.3, epochs=20, callbacks=[early_stopping_monitor])

Now, its possible to make perdiction with the model with unseen data using .predict() method.

In [ ]:
# Generate data for prediction
pred_data = np.array([[  2.      ,  34.      ,   0.      ,   0.      ,  13.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  31.      ,   1.      ,   1.      ,  26.25    ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  11.      ,   1.      ,   2.      , 120.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,   0.42    ,   0.      ,   1.      ,   8.5167  ,
          1.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  27.      ,   0.      ,   0.      ,   6.975   ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  31.      ,   0.      ,   0.      ,   7.775   ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  39.      ,   0.      ,   0.      ,   0.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  18.      ,   0.      ,   0.      ,   7.775   ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  39.      ,   0.      ,   0.      ,  13.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  33.      ,   1.      ,   0.      ,  53.1     ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  26.      ,   0.      ,   0.      ,   7.8875  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  39.      ,   0.      ,   0.      ,  24.15    ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  35.      ,   0.      ,   0.      ,  10.5     ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,   6.      ,   4.      ,   2.      ,  31.275   ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  30.5     ,   0.      ,   0.      ,   8.05    ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  29.699118,   0.      ,   0.      ,   0.      ,
          1.      ,   1.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  23.      ,   0.      ,   0.      ,   7.925   ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  31.      ,   1.      ,   1.      ,  37.0042  ,
          1.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  43.      ,   0.      ,   0.      ,   6.45    ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  10.      ,   3.      ,   2.      ,  27.9     ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  52.      ,   1.      ,   1.      ,  93.5     ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  27.      ,   0.      ,   0.      ,   8.6625  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  38.      ,   0.      ,   0.      ,   0.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  27.      ,   0.      ,   1.      ,  12.475   ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,   2.      ,   4.      ,   1.      ,  39.6875  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  29.699118,   0.      ,   0.      ,   6.95    ,
          1.      ,   1.      ,   0.      ,   1.      ,   0.      ],
       [  3.      ,  29.699118,   0.      ,   0.      ,  56.4958  ,
          1.      ,   1.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,   1.      ,   0.      ,   2.      ,  37.0042  ,
          1.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  29.699118,   0.      ,   0.      ,   7.75    ,
          1.      ,   1.      ,   0.      ,   1.      ,   0.      ],
       [  1.      ,  62.      ,   0.      ,   0.      ,  80.      ,
          0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
       [  3.      ,  15.      ,   1.      ,   0.      ,  14.4542  ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  2.      ,   0.83    ,   1.      ,   1.      ,  18.75    ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  29.699118,   0.      ,   0.      ,   7.2292  ,
          1.      ,   1.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  23.      ,   0.      ,   0.      ,   7.8542  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  18.      ,   0.      ,   0.      ,   8.3     ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  39.      ,   1.      ,   1.      ,  83.1583  ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  21.      ,   0.      ,   0.      ,   8.6625  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  29.699118,   0.      ,   0.      ,   8.05    ,
          1.      ,   1.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  32.      ,   0.      ,   0.      ,  56.4958  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  29.699118,   0.      ,   0.      ,  29.7     ,
          1.      ,   1.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  20.      ,   0.      ,   0.      ,   7.925   ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  16.      ,   0.      ,   0.      ,  10.5     ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  30.      ,   0.      ,   0.      ,  31.      ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  34.5     ,   0.      ,   0.      ,   6.4375  ,
          1.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  17.      ,   0.      ,   0.      ,   8.6625  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  42.      ,   0.      ,   0.      ,   7.55    ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  29.699118,   8.      ,   2.      ,  69.55    ,
          1.      ,   1.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  35.      ,   0.      ,   0.      ,   7.8958  ,
          1.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  2.      ,  28.      ,   0.      ,   1.      ,  33.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  29.699118,   1.      ,   0.      ,  89.1042  ,
          0.      ,   1.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,   4.      ,   4.      ,   2.      ,  31.275   ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  74.      ,   0.      ,   0.      ,   7.775   ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,   9.      ,   1.      ,   1.      ,  15.2458  ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  1.      ,  16.      ,   0.      ,   1.      ,  39.4     ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  44.      ,   1.      ,   0.      ,  26.      ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  18.      ,   0.      ,   1.      ,   9.35    ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  45.      ,   1.      ,   1.      , 164.8667  ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  51.      ,   0.      ,   0.      ,  26.55    ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  24.      ,   0.      ,   3.      ,  19.2583  ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  29.699118,   0.      ,   0.      ,   7.2292  ,
          1.      ,   1.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  41.      ,   2.      ,   0.      ,  14.1083  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  21.      ,   1.      ,   0.      ,  11.5     ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  48.      ,   0.      ,   0.      ,  25.9292  ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  29.699118,   8.      ,   2.      ,  69.55    ,
          0.      ,   1.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  24.      ,   0.      ,   0.      ,  13.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  42.      ,   0.      ,   0.      ,  13.      ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  27.      ,   1.      ,   0.      ,  13.8583  ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  1.      ,  31.      ,   0.      ,   0.      ,  50.4958  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  29.699118,   0.      ,   0.      ,   9.5     ,
          1.      ,   1.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,   4.      ,   1.      ,   1.      ,  11.1333  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  26.      ,   0.      ,   0.      ,   7.8958  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  47.      ,   1.      ,   1.      ,  52.5542  ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  33.      ,   0.      ,   0.      ,   5.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  47.      ,   0.      ,   0.      ,   9.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  28.      ,   1.      ,   0.      ,  24.      ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  15.      ,   0.      ,   0.      ,   7.225   ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  20.      ,   0.      ,   0.      ,   9.8458  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  19.      ,   0.      ,   0.      ,   7.8958  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  29.699118,   0.      ,   0.      ,   7.8958  ,
          1.      ,   1.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  56.      ,   0.      ,   1.      ,  83.1583  ,
          0.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  2.      ,  25.      ,   0.      ,   1.      ,  26.      ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  33.      ,   0.      ,   0.      ,   7.8958  ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  22.      ,   0.      ,   0.      ,  10.5167  ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  2.      ,  28.      ,   0.      ,   0.      ,  10.5     ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  25.      ,   0.      ,   0.      ,   7.05    ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  39.      ,   0.      ,   5.      ,  29.125   ,
          0.      ,   0.      ,   0.      ,   1.      ,   0.      ],
       [  2.      ,  27.      ,   0.      ,   0.      ,  13.      ,
          1.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  19.      ,   0.      ,   0.      ,  30.      ,
          0.      ,   0.      ,   0.      ,   0.      ,   1.      ],
       [  3.      ,  29.699118,   1.      ,   2.      ,  23.45    ,
          0.      ,   1.      ,   0.      ,   0.      ,   1.      ],
       [  1.      ,  26.      ,   0.      ,   0.      ,  30.      ,
          1.      ,   0.      ,   1.      ,   0.      ,   0.      ],
       [  3.      ,  32.      ,   0.      ,   0.      ,   7.75    ,
          1.      ,   0.      ,   0.      ,   1.      ,   0.      ]])

In [ ]:
# Calculate predictions: predictions
predictions = model.predict(pred_data)

# Calculate predicted probability of survival: predicted_prob_true
predicted_prob_true = predictions[:, 1]

# Print predicted_prob_true
print(predicted_prob_true)

In [ ]:
# summary of the mddel
model.summary()